In [126]:
import pandas as pd
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
from selenium import webdriver
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, MetaData, Column, Integer, Table
import sqlite3 as sq
import time

In [127]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [128]:
url = 'https://mbawa1139.github.io/ETL-Project/motorcycle_accident.html'
browser.visit(url)

In [129]:
html = browser.html

soup = BeautifulSoup(html, 'html.parser')

In [130]:
# reads and retrieves all tables in site
table_df = pd.read_html(url)

# Selecting the desired table (first one)
table_df = table_df[0]

# Changed the index of the table to the first column
#table_df.set_index("Accident_Index", inplace = True)
table_df

,Unnamed: 0,Accident_Index,Vehicle_Type,Longitude,Latitude,Sex_of_Casualty,Age_of_Casualty
0,0,201001BS70008,Motorcycle,-0.175802,51.484361,Male,28
1,1,201001BS70011,Motorcycle,-0.217699,51.514321,Male,21
2,2,201001BS70016,Motorcycle,-0.173157,51.485669,Male,14
3,3,201001BS70016,Motorcycle,-0.173157,51.485669,Male,-1
4,4,201001BS70020,Motorcycle,-0.195276,51.494554,Male,23
...,...,...,...,...,...,...,...
114277,114277,2014984120014,Motorcycle,-3.261808,54.987381,Male,25
114278,114278,2014984121914,Motorcycle,-3.507553,55.408030,Male,59
114279,114279,2014984127714,Motorcycle,-3.479810,55.118296,Male,42
114280,114280,2014984129614,Motorcycle,-3.369973,55.352170,Male,35


In [131]:
# Remove rows with age value of -1
clean_table_df = table_df[table_df["Age_of_Casualty"] != -1]
clean_table_df

,Unnamed: 0,Accident_Index,Vehicle_Type,Longitude,Latitude,Sex_of_Casualty,Age_of_Casualty
0,0,201001BS70008,Motorcycle,-0.175802,51.484361,Male,28
1,1,201001BS70011,Motorcycle,-0.217699,51.514321,Male,21
2,2,201001BS70016,Motorcycle,-0.173157,51.485669,Male,14
4,4,201001BS70020,Motorcycle,-0.195276,51.494554,Male,23
5,5,201001BS70022,Motorcycle,-0.185327,51.494761,Male,37
...,...,...,...,...,...,...,...
114277,114277,2014984120014,Motorcycle,-3.261808,54.987381,Male,25
114278,114278,2014984121914,Motorcycle,-3.507553,55.408030,Male,59
114279,114279,2014984127714,Motorcycle,-3.479810,55.118296,Male,42
114280,114280,2014984129614,Motorcycle,-3.369973,55.352170,Male,35


In [23]:
# Remove
clean_table_df = clean_table_df.drop(clean_table_df.columns[0], axis = 1)
clean_table_df

,Accident_Index,Vehicle_Type,Longitude,Latitude,Sex_of_Casualty,Age_of_Casualty
0,201001BS70008,Motorcycle,-0.175802,51.484361,Male,28
1,201001BS70011,Motorcycle,-0.217699,51.514321,Male,21
2,201001BS70016,Motorcycle,-0.173157,51.485669,Male,14
4,201001BS70020,Motorcycle,-0.195276,51.494554,Male,23
5,201001BS70022,Motorcycle,-0.185327,51.494761,Male,37
...,...,...,...,...,...,...
114277,2014984120014,Motorcycle,-3.261808,54.987381,Male,25
114278,2014984121914,Motorcycle,-3.507553,55.408030,Male,59
114279,2014984127714,Motorcycle,-3.479810,55.118296,Male,42
114280,2014984129614,Motorcycle,-3.369973,55.352170,Male,35


In [132]:
# Add year column that comes from first for of Accident Index
clean_table_df["Year"] = clean_table_df["Accident_Index"].str[:4]

# Organize Columns
clean_table_df = clean_table_df[['Accident_Index', 'Year','Vehicle_Type', 'Longitude', 
    'Latitude', 'Sex_of_Casualty', 'Age_of_Casualty']]

clean_table_df

/Users/arselvargas/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Accident_Index,Year,Vehicle_Type,Longitude,Latitude,Sex_of_Casualty,Age_of_Casualty
0,201001BS70008,2010,Motorcycle,-0.175802,51.484361,Male,28
1,201001BS70011,2010,Motorcycle,-0.217699,51.514321,Male,21
2,201001BS70016,2010,Motorcycle,-0.173157,51.485669,Male,14
4,201001BS70020,2010,Motorcycle,-0.195276,51.494554,Male,23
5,201001BS70022,2010,Motorcycle,-0.185327,51.494761,Male,37
...,...,...,...,...,...,...,...
114277,2014984120014,2014,Motorcycle,-3.261808,54.987381,Male,25
114278,2014984121914,2014,Motorcycle,-3.507553,55.408030,Male,59
114279,2014984127714,2014,Motorcycle,-3.479810,55.118296,Male,42
114280,2014984129614,2014,Motorcycle,-3.369973,55.352170,Male,35


In [133]:
# Create the decade bins 
bins = [0, 17, 25, 29, 39, 49, 60]

# Create the names for the bins
group_names = ["Under 18", "18 to 25", "26 to 35", "36 to 45", "46 to 55", "Over 55"]

# coverting year column datatype to interger
clean_table_df['Age_of_Casualty'] = clean_table_df['Age_of_Casualty'].astype(int)

# add bin column to dataframe
clean_table_df["Age_Group"] = pd.cut(clean_table_df["Age_of_Casualty"], bins, labels=group_names, include_lowest=True)

# reorganize columns
clean_table_df = clean_table_df[['Accident_Index', 'Year','Vehicle_Type', 'Longitude', 
         'Latitude', 'Sex_of_Casualty', 'Age_of_Casualty', 'Age_Group']]

clean_table_df

/Users/arselvargas/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Accident_Index,Year,Vehicle_Type,Longitude,Latitude,Sex_of_Casualty,Age_of_Casualty,Age_Group
0,201001BS70008,2010,Motorcycle,-0.175802,51.484361,Male,28,26 to 35
1,201001BS70011,2010,Motorcycle,-0.217699,51.514321,Male,21,18 to 25
2,201001BS70016,2010,Motorcycle,-0.173157,51.485669,Male,14,Under 18
4,201001BS70020,2010,Motorcycle,-0.195276,51.494554,Male,23,18 to 25
5,201001BS70022,2010,Motorcycle,-0.185327,51.494761,Male,37,36 to 45
...,...,...,...,...,...,...,...,...
114277,2014984120014,2014,Motorcycle,-3.261808,54.987381,Male,25,18 to 25
114278,2014984121914,2014,Motorcycle,-3.507553,55.408030,Male,59,Over 55
114279,2014984127714,2014,Motorcycle,-3.479810,55.118296,Male,42,46 to 55
114280,2014984129614,2014,Motorcycle,-3.369973,55.352170,Male,35,36 to 45


In [134]:
# Path to SQLite file
sql_lite = "sqlite:///uk_motorcycle_accidents_sqlite.sqlite"

# Initial DB connection
engine = create_engine(sql_lite)

In [158]:
sql_lite = "sqlite:///uk_motorcycle_accidents_sqlite.db"

# Initial DB connection
engine = create_engine(sql_lite)

In [135]:
# Read DF into DB
clean_table_df.to_sql(name='uk_motorcycle_accidents', con=engine, if_exists='append', index=False)

In [159]:
# Check connection of DB is working
engine.table_names()

['uk_motorcycle_accidents']

In [140]:
# Tried setting up a key for the table
metadata = MetaData()
moto_tbl = Table('uk_motorcycle_accidents',metadata, Column('Accident_Index', Integer, 
primary_key=True), autoload=True, autoload_with=engine)
Base.prepare()
#view_name = Base.classes.uk_motorcycle_accidents


In [141]:
# Reflect Database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)


In [142]:
Base.classes.keys()

[]

In [160]:
pd.read_sql_query('Select * from uk_motorcycle_accidents', con=engine).head()

,Accident_Index,Year,Vehicle_Type,Longitude,Latitude,Sex_of_Casualty,Age_of_Casualty,Age_Group
0,201001BS70008,2010,Motorcycle,-0.175802,51.484361,Male,28,26 to 35
1,201001BS70011,2010,Motorcycle,-0.217699,51.514321,Male,21,18 to 25
2,201001BS70016,2010,Motorcycle,-0.173157,51.485669,Male,14,Under 18
3,201001BS70020,2010,Motorcycle,-0.195276,51.494554,Male,23,18 to 25
4,201001BS70022,2010,Motorcycle,-0.185327,51.494761,Male,37,36 to 45


In [161]:
# Checking for table columns
inspector = inspect(engine)
casualty_columns = inspector.get_columns('uk_motorcycle_accidents')
for c in casualty_columns:
    print(c['name'], c["type"])

Accident_Index TEXT
Year TEXT
Vehicle_Type TEXT
Longitude FLOAT
Latitude FLOAT
Sex_of_Casualty TEXT
Age_of_Casualty BIGINT
Age_Group TEXT


In [169]:
session = Session(engine)

In [162]:
con.execute('Select * From uk_motorcycle_accidents LIMIT 10').fetchall()

[('201001BS70008', '2010', 'Motorcycle', -0.17580199999999999, 51.484361, 'Male', 28, '26 to 35'),
 ('201001BS70011', '2010', 'Motorcycle', -0.21769899999999998, 51.514321, 'Male', 21, '18 to 25'),
 ('201001BS70016', '2010', 'Motorcycle', -0.173157, 51.485669, 'Male', 14, 'Under 18'),
 ('201001BS70020', '2010', 'Motorcycle', -0.195276, 51.494554, 'Male', 23, '18 to 25'),
 ('201001BS70022', '2010', 'Motorcycle', -0.185327, 51.494761, 'Male', 37, '36 to 45'),
 ('201001BS70022', '2010', 'Motorcycle', -0.185327, 51.494761, 'Male', 69, None),
 ('201001BS70025', '2010', 'Motorcycle', -0.18446400000000002, 51.483866, 'Female', 78, None),
 ('201001BS70027', '2010', 'Motorcycle', -0.173644, 51.495119, 'Female', 34, '36 to 45'),
 ('201001BS70031', '2010', 'Motorcycle', -0.16705699999999998, 51.497714, 'Female', 34, '36 to 45'),
 ('201001BS70033', '2010', 'Motorcycle', -0.19728900000000002, 51.490988, 'Male', 32, '36 to 45')]

In [168]:
engine.execute('Select Longitude From uk_motorcycle_accidents Where Age_Group = "18 to 25" LIMIT 10').fetchall()

[(-0.21769899999999998,),
 (-0.195276,),
 (-0.195845,),
 (-0.179153,),
 (-0.163523,),
 (-0.184044,),
 (-0.191117,),
 (-0.19578299999999998,),
 (-0.18431199999999998,),
 (-0.158521,)]

In [170]:
# Good example of querying the database and assigning to variable
coord = con.execute('Select Sex_of_Casualty, Year,Latitude, Longitude From uk_motorcycle_accidents Where Age_Group = "18 to 25" LIMIT 10').fetchall()
print(coord)

[('Male', '2010', 51.514321, -0.21769899999999998), ('Male', '2010', 51.494554, -0.195276), ('Male', '2010', 51.494743, -0.195845), ('Male', '2010', 51.487021, -0.179153), ('Male', '2010', 51.492443, -0.163523), ('Male', '2010', 51.501665, -0.184044), ('Male', '2010', 51.490444000000004, -0.191117), ('Female', '2010', 51.499958, -0.19578299999999998), ('Male', '2010', 51.484044, -0.18431199999999998), ('Female', '2010', 51.502165999999995, -0.158521)]


In [119]:
# Doesn't work no key
last_accident = session.query(uk_motorcycle_accidents.Accident_Index).order_by(uk_motorcycle_accidents.Accident_Index.desc()).first()
print(last_accident)

NameError: name 'uk_motorcycle_accidents' is not defined

In [85]:

first_ag = session.query(uk_motorcycle_accidents.Year).filter(Age_Group="Under 18").count
print(first_ag)

NameError: name 'uk_motorcycle_accidents' is not defined

In [147]:
pd.read_sql_query('Select * from uk_motorcycle_accidents', con=engine).head()

,Accident_Index,Year,Vehicle_Type,Longitude,Latitude,Sex_of_Casualty,Age_of_Casualty,Age_Group
0,201001BS70008,2010,Motorcycle,-0.175802,51.484361,Male,28,26 to 35
1,201001BS70011,2010,Motorcycle,-0.217699,51.514321,Male,21,18 to 25
2,201001BS70016,2010,Motorcycle,-0.173157,51.485669,Male,14,Under 18
3,201001BS70020,2010,Motorcycle,-0.195276,51.494554,Male,23,18 to 25
4,201001BS70022,2010,Motorcycle,-0.185327,51.494761,Male,37,36 to 45


In [86]:
Year = Base.classes.Year

AttributeError: Year

In [157]:
session.close()

In [73]:
sql_lite = "sqlite:///uk_motorcycle_accidents_sqlite.sqlite"

# Initial DB connection
engine = create_engine(sql_lite)
con = engine.connect()

In [71]:
#con = sq.connect("sqlite:///uk_motorcycle_accidents_sqlite.sqlite")

OperationalError: unable to open database file